In [24]:
import requests
import xml.etree.ElementTree as ET 
import xml.sax
import pandas as pd

In [25]:
Keys = [0] * 13
country = 'USA'
year = 2012
month = 12
term_length = 4

**Keys 1: Party Mandate**

Key 1 is turned True if the incumbent Party has had a net gain of seats from the previous Presidential and Midterm elections combined. In other words, does the incumbent party hold more seats than they held after the midterm elections of the previous presidency?

In 2012, the Democrats held 193 seats versus 233 in in 2006. Therefore, the key is turned False.

In [26]:
Keys[0] = 0

**Keys 2, 3, 4: Primary Contest, Incumbent President, Third Party**

Keys 2, 3, and 4 are difficult to quantify, and while be up to the user to determine. However, these should be quite objective, since keys 2, 3, and 4 are:

- No Primary Contest

- Incumbent Seeking Re-Election

- No Third Party

In the 2024 Election, these were the respective results for each of the given keys.

No Primary Contest - True. Obama secured the Democratic nomination without contest. 

Incumbent Seeking Re-Election - True. Obama was already president.

No Third Party - In Lichtman's model, this key is turned true if there is a third party candidate polling consistently above 10% in the polls. High support for a third party indicates dissatisfaction with the incumbent party. In 2012, this was True.

In [27]:
Keys[1] = 1
Keys[2] = 1
Keys[3] = 1

**Keys 5: Short Term Economy**

Is the country in a recession during the election campaign? 

A Recession is two consecutive quarters of GDP decline. 

-> Check average GDP of three quarters, check if decreasing -> Recession

In [28]:
GDP = pd.read_excel(country + ".xlsx", sheet_name= 'Quarterly')

GDP.columns = ['Date', 'GDP']

GDP.index = GDP.Date

GDP.drop(columns = 'Date', inplace=True)

display(GDP)

,GDP
Date,
1947-01-01,2169.189
1947-04-01,2167.121
1947-07-01,2169.565
1947-10-01,2190.947
1948-01-01,2250.560
...,...
2023-07-01,22602.433
2023-10-01,22880.288
2024-01-01,23035.857


In [29]:
months = pd.DataFrame()

quarters_taken = 8

take_month = month - 9

if take_month == 1:
    starting_month = '-12'
    start_year = year - 2

elif take_month - 1 < 10:
    starting_month = '-0' + str(take_month - 1)
    start_year = year -1
else:
    starting_month = '-' + str(take_month - 1)
    start_year = year - 1


months['GDP'] = GDP[(GDP.index >= str(start_year) + starting_month) & (GDP.index <= str(year+1))]

months['GDP Growth'] = months.GDP.pct_change().ffill()
months.drop(index=months.index[0], inplace=True)

display(months)

,GDP,GDP Growth
Date,,
2011-07-01,17350.022,0.003518
2011-10-01,17522.750,0.009955
2012-01-01,17848.839,0.018609
2012-04-01,17869.339,0.001149
2012-07-01,17719.616,-0.008379
2012-10-01,17966.932,0.013957


In [30]:
decreased_month = 0
recession = False

for i in range(1, len(months.index)):
    ind = months.index[i]

    if months.loc[ind, 'GDP Growth'] < 0:
        decreased_month += 1

        if decreased_month >= 2:
            recession = True
            i = len(months.index)

    else:
        decreased_month = 0

if recession:
    print('Key 5 is False. There was a recession during the election campaign.')
    

else:
    print('Key 5 is True. There was no recession during the election campaign.')
    Keys[4] = 1

Key 5 is True. There was no recession during the election campaign.


**Key 6: Long Term Economy**

Real per Capita economic growth during term >= mean growth during previous 2 terms means that strong economic growth

In [31]:


#last two terms
start = year - 13

GDP_api = "https://api.worldbank.org/v2/country/" + country + "/indicator/NY.GDP.PCAP.CD" + '?date=' + str(start) + ':' + str(year)

inflation_api = "https://api.worldbank.org/v2/country/" + country + "/indicator/NY.GDP.DEFL.KD.ZG" + '?date=' + str(start) + ':' + str(year)

gdp_file = requests.get(GDP_api).text

inflation_file = requests.get(inflation_api).text

print(inflation_file)

ï»¿<?xml version="1.0" encoding="utf-8"?>
<wb:data page="1" pages="1" per_page="50" total="14" sourceid="2" lastupdated="2024-12-16" xmlns:wb="http://www.worldbank.org">
  <wb:data>
    <wb:indicator id="NY.GDP.DEFL.KD.ZG">Inflation, GDP deflator (annual %)</wb:indicator>
    <wb:country id="US">United States</wb:country>
    <wb:countryiso3code>USA</wb:countryiso3code>
    <wb:date>2012</wb:date>
    <wb:value>1.8621649281277</wb:value>
    <wb:unit />
    <wb:obs_status />
    <wb:decimal>1</wb:decimal>
  </wb:data>
  <wb:data>
    <wb:indicator id="NY.GDP.DEFL.KD.ZG">Inflation, GDP deflator (annual %)</wb:indicator>
    <wb:country id="US">United States</wb:country>
    <wb:countryiso3code>USA</wb:countryiso3code>
    <wb:date>2011</wb:date>
    <wb:value>2.06310947543149</wb:value>
    <wb:unit />
    <wb:obs_status />
    <wb:decimal>1</wb:decimal>
  </wb:data>
  <wb:data>
    <wb:indicator id="NY.GDP.DEFL.KD.ZG">Inflation, GDP deflator (annual %)</wb:indicator>
    <wb:country id

In [32]:
def calculate_inflation(rates):
    inflation = pd.DataFrame(columns = ['inflation'], index = rates.index)
    
    for i in inflation.index:    
        inflation.loc[i, 'inflation'] = 1
        
        for j in range(int(rates.index[0]), int(i)):
            inflation.loc[i, 'inflation']  += rates[str(j)]

            #print(rates[str(j)])

    return inflation

In [33]:
country_data = pd.DataFrame(columns = ['Date', 'Value', 'inflation', 'Real Value', 'Real Increase'])



class XMLHandler(xml.sax.ContentHandler):
    
    def __init__(self):
        self.CurrentData = ""
        self.country = ""
        self.date = ""
        self.value = ""
        self.count = 0

    def startElement(self, tag, attribute):
        self.CurrentData = tag

        if tag == "wb:data":
            self.count += 1

    def endElement(self, tag):
        
        if self.CurrentData == "wb:date":

            country_data.loc[self.count, 'Date'] = self.date
        
        elif self.CurrentData == "wb:value":
            
            country_data.loc[self.count, 'Value'] = float(self.value)

    def characters(self, content):
        if self.CurrentData == "wb:date":
            self.date = content
        elif self.CurrentData == "wb:value":
            self.value = content

parser = xml.sax.make_parser()

parser.setFeature(xml.sax.handler.feature_namespaces, 0)

Handler = XMLHandler()

parser.setContentHandler( Handler )
parser.parse(GDP_api)

country_data.index = country_data.Date

country_data.drop(columns = 'Date', inplace = True)

country_data.sort_values(by='Date', ascending=True, inplace=True)

gdp_data = country_data.copy()

country_data = pd.DataFrame(columns = ['Date', 'Value'])

parser.parse(inflation_api)

country_data.index = country_data.Date

country_data.drop(columns = 'Date', inplace = True)

country_data.sort_values(by='Date', ascending=True, inplace=True)

inflation_data = country_data/100

gdp_data.inflation = calculate_inflation(inflation_data.Value)

display(gdp_data)

,Value,inflation,Real Value,Real Increase
Date,,,,
1999,34515.381307,1,NaN,NaN
2000,36329.97026,1.014151,NaN,NaN
2001,37133.620397,1.036803,NaN,NaN
2002,37997.74243,1.05932,NaN,NaN
2003,39490.30239,1.07486,NaN,NaN
2004,41724.641198,1.094603,NaN,NaN
2005,44123.399647,1.121494,NaN,NaN
2006,46301.987649,1.152848,NaN,NaN
2007,48050.227412,1.183687,NaN,NaN


In [34]:
gdp_data['Real Value'] = gdp_data.Value/gdp_data.inflation

gdp_data['Real Increase'] = gdp_data['Real Value'].pct_change().ffill()

gdp_data.drop(index=gdp_data.index[0], inplace=True)

display(gdp_data)

C:\Users\paliu\AppData\Local\Temp\ipykernel_27252\1300146166.py:3: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  gdp_data['Real Increase'] = gdp_data['Real Value'].pct_change().ffill()


,Value,inflation,Real Value,Real Increase
Date,,,,
2000,36329.97026,1.014151,35823.039108,0.037886
2001,37133.620397,1.036803,35815.504777,-0.000210
2002,37997.74243,1.05932,35869.94341,0.001520
2003,39490.30239,1.07486,36739.957316,0.024255
2004,41724.641198,1.094603,38118.517882,0.037522
2005,44123.399647,1.121494,39343.418196,0.032134
2006,46301.987649,1.152848,40163.120982,0.020835
2007,48050.227412,1.183687,40593.703398,0.010721
2008,48570.059427,1.210779,40114.709134,-0.011800


In [35]:
#Now calculate the average increase for each term

year = int(gdp_data.index[0])

avg_increases = [0] * 3

for i in range(len(avg_increases)):
    for j in range(term_length):
        avg_increases[i] += gdp_data.loc[str(year), "Real Increase"]
        year += 1

    avg_increases[i] /= term_length

#print(avg_increases)

pre_term_increase = avg_increases[0] + avg_increases[1] 
pre_term_increase /= 2

if pre_term_increase < avg_increases[-1]:
    print("Key 6 is True. The incumbent party has had a strong long term economy")
    Keys[5] = 1

else:
    print("Key 6 is False. The incumbent party has had a weak long term economy")
    Keys[5] = 0

Key 6 is False. The incumbent party has had a weak long term economy


**Key 7: Policy Change**

To determine this, we will analyze the number of legislative bills passed through Congress throughout the two half-terms of the current presidency. If this is less than 1 standard deviation of the mean number of bills passed, then the term has not brought about a major policy change.

Data was drawn from: https://www.govtrack.us/congress/bills/statistics 

In [36]:
Bills = pd.read_excel(country + "-bills.xlsx", sheet_name= 'Quarterly')

Bills.columns = ['Date', 'End', 'Bills']

Bills.index = Bills.Date

Bills.drop(columns = 'Date', inplace=True)


#Compare to the previous 10 presidencies

start = year - 10 * 4

values_used = Bills[(Bills.index > start) & (Bills.index <= year)]

display(values_used)

,End,Bills
Date,,
1973,1974,772
1975,1976,729
1977,1978,804
1979,1980,736
1981,1982,529
1983,1984,677
1985,1986,687
1987,1988,761
1989,1990,665


In [37]:
bills_by_presidency = pd.DataFrame(columns = ['Bills'])

for i in range(len(values_used.index)):
    if i % 2 == 0:
        bills_by_presidency.loc[values_used.index[i], 'Bills'] = values_used.loc[values_used.index[i], 'Bills']

    else:
        bills_by_presidency.loc[values_used.index[i-1], 'Bills'] += values_used.loc[values_used.index[i], 'Bills']

display(bills_by_presidency)

,Bills
1973,1501
1977,1540
1981,1206
1985,1448
1989,1275
1993,810
1997,1008
2001,887
2005,943
2009,669


In [38]:
if bills_by_presidency.Bills[bills_by_presidency.index[-1]] < bills_by_presidency.Bills.mean() - bills_by_presidency.Bills.std():
    print("There was no major policy change.")
    print("Key 7 is False.")
    Keys[6] = 0

else:
    print("There was a major policy change.")
    print("Key 7 is True.")
    Keys[6] = 1

There was no major policy change.
Key 7 is False.


**Key 8: Social Unrest**

Has there been social unrest during the previous term? If it is low, then the key is True, otherwise the key is False and there is risk of the incumbent party losing power.

We will use the World Bank's measure for Political Stability and Absence of Violence/Terrorism. If there is any term where the measure was 1 std below the mean measure, then that term is deemed to have had social unrest and therefore, the key is turned False.

In [39]:
start = year -10

unrest_api = "https://api.worldbank.org/v2/country/" + country + "/indicator/PV.EST" + '?date=' + str(start) + ':' + str(year)

In [40]:


country_data = pd.DataFrame(columns = ['Date', 'Value'])

parser.parse(unrest_api)
country_data.index = country_data.Date

country_data.drop(columns = 'Date', inplace = True)

country_data.sort_values(by='Date', ascending=True, inplace=True)

unrest_data = country_data

display(unrest_data)

,Value
Date,
2002,0.285477
2003,0.08044
2004,-0.23304
2005,-0.060543
2006,0.490835
2007,0.374478
2008,0.58266
2009,0.447314
2010,0.438404


In [41]:
#Take the average of the last 10 years
unrest_avg = unrest_data.Value.mean()
unrest_std = unrest_data.Value.std()

print(unrest_avg)
print(unrest_std)

unrest = False

display(unrest_data)

for i in range(year - term_length + 1, year):
    
    if unrest_data.Value[str(i)] < unrest_avg - unrest_std:
        unrest = True
        print("There was unrest in", i)
        print("Key 8 is False.")
        Keys[7] = 0

if not unrest:
    print("There was no unrest.")
    print("Key 8 is True.")
    Keys[7] = 1

0.3299740163440054
0.28473121591762485


,Value
Date,
2002,0.285477
2003,0.08044
2004,-0.23304
2005,-0.060543
2006,0.490835
2007,0.374478
2008,0.58266
2009,0.447314
2010,0.438404


There was no unrest.
Key 8 is True.


**Key 9: Scandal**

Has there been a major scandal during the previous term? If it is low, then the key is True, otherwise the key is False and there is risk of the incumbent party losing power.

We will use the World Bank's measure fosm. If there is any term where the measure was 1 std below the mean measure, then that term is deemed to have had social unrest and therefore, the key is turned False.

In [42]:
start = year -10

unrest_api = "https://api.worldbank.org/v2/country/" + country + "/indicator/CC.EST" + '?date=' + str(start) + ':' + str(year)

In [43]:
country_data = pd.DataFrame(columns = ['Date', 'Value'])

parser.parse(unrest_api)
country_data.index = country_data.Date

country_data.drop(columns = 'Date', inplace = True)

country_data.sort_values(by='Date', ascending=True, inplace=True)

unrest_data = country_data

display(unrest_data)

,Value
Date,
2002,1.873597
2003,1.704491
2004,1.810702
2005,1.536727
2006,1.344893
2007,1.377069
2008,1.437633
2009,1.286476
2010,1.265258


In [44]:
#Take the average of the last 10 years
unrest_avg = unrest_data.Value.mean()
unrest_std = unrest_data.Value.std()

print(unrest_avg)
print(unrest_std)

unrest = False

display(unrest_data)

for i in range(year - term_length + 1, year):
    
    if unrest_data.Value[str(i)] < unrest_avg - unrest_std:
        unrest = True
        print("There was high corruption in", i)
        print("Key 9 is False.")
        Keys[8] = 0

if not unrest:
    print("There was low corruption.")
    print("Key 9 is True.")
    Keys[8] = 1

1.4819780804894185
0.22033952756173267


,Value
Date,
2002,1.873597
2003,1.704491
2004,1.810702
2005,1.536727
2006,1.344893
2007,1.377069
2008,1.437633
2009,1.286476
2010,1.265258


There was low corruption.
Key 9 is True.


**Keys 10 and 11**

Keys 10 and 11 deals with foreign and military matters. Success in diplomacy is also difficult to quantify, so these two keys are also subjective. 

With regards to the 2012 election, Bin Laden was killed during Obama's administration, so Key 10 is True. There was no major foreign failure either, so Key 11 is True.

In [45]:
Keys[10] = 1
Keys[11] = 1

**Keys 12 and 13**

These last two keys, candidate charisma, are by far the most subjective. 

Key 12: Is the incumbent party candidate charismatic or a national hero? Obama was considered charismatic, so this is True.

Key 13: Is the challenging party candidate charismatic or a national hero? Romney was not particularly charismatic, so this is True.

In [46]:
Keys[11] = 1
Keys[12] = 0

In [47]:
print(Keys)

[0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0]


In [50]:
keys_true = 0

for i in Keys:
    keys_true += i

if keys_true >= 8:
    print("The incumbent party will win")
else:
    print("The challenging party will win")

The incumbent party will win
